In [29]:
from dash import Dash, dcc, html, Input, Output, callback
import requests

app = Dash(__name__)

ml_serving_url = "https://clearml-serving.internal.magiccityit.com/serve"
epa_path = "/epa_model/"
wp_path = "/vegas_wp_model/"

app.layout = html.Div([
    html.H2("Spread-Based Win Probability Calculator"),
    html.Div(["Game ID: ", dcc.Input(id='game_id', value="2023_PIT_BUF", type='text')]),
    html.Div(["Home Team: ", dcc.Input(id='home_team_wp', value='PIT', type='text')]),
    html.Div(["Away Team: ", dcc.Input(id='away_team_wp', value='BUF', type='text')]),
    html.Div(["Possessing Team: ", dcc.Input(id='posteam_wp', value="PIT", type='text')]),
    html.Div(["Defending Team: ", dcc.Input(id='defteam_wp', value="BUF", type='text')]),
    html.Div(["Possessing Team Timeouts Remaining: ", dcc.Input(id='posteam_timeouts_remaining_wp', value=3, type='number')]),
    html.Div(["Defending Team Timeouts Remaining: ", dcc.Input(id='defteam_timeouts_remaining_wp', value=3, type='number')]),
    html.Div(["Seconds Remaining in the Game: ", dcc.Input(id='game_seconds_remaining', value=3600, type='number')]),
    html.Div(["Seconds Remaining this Half: ", dcc.Input(id='half_seconds_remaining_wp', value=1800, type='number')]),
    html.Div(["Quarter: ", dcc.Input(id='qtr', value=1, type='number')]),
    html.Div(["Yardline Position: ", dcc.Input(id='yardline_100_wp', value=75, type='number')]),
    html.Div(["Down: ", dcc.Input(id='down_wp', value=1, type='number')]),
    html.Div(["Yards to Go: ", dcc.Input(id='ydstogo_wp', value=10, type='number')]),
    html.Div(["Spread Line: ", dcc.Input(id='spread_line', value=0, type='number')]),
    html.Div(["Score Differential: ", dcc.Input(id='score_differential', value=-3, type='number')]),
    html.Div(["Result: ", dcc.Input(id='result', value=0, type='number')]),
    html.Br(),
    html.Div(id="home_win_prob"),
    html.Div(id="away_win_prob"),
    html.Br(),
    html.H2("EPA Calculator"),
    html.Div(["Season: ", dcc.Input(id='season', value=2023, type='number')]),
    html.Div(["Home Team: ", dcc.Input(id='home_team', value='PIT', type='text')]),
    html.Div(["Away Team: ", dcc.Input(id='away_team', value='BUF', type='text')]),
    html.Div(["Possessing Team: ", dcc.Input(id='posteam', value="PIT", type='text')]),
    html.Div(["Defending Team: ", dcc.Input(id='defteam', value="BUF", type='text')]),
    html.Div(["Possessing Team Timeouts Remaining: ", dcc.Input(id='posteam_timeouts_remaining', value=3, type='number')]),
    html.Div(["Defending Team Timeouts Remaining: ", dcc.Input(id='defteam_timeouts_remaining', value=3, type='number')]),
    html.Div(["Seconds Remaining this Half: ", dcc.Input(id='half_seconds_remaining', value=1800, type='number')]),
    html.Div(["Yardline Position: ", dcc.Input(id='yardline_100', value=75, type='number')]),
    html.Div(["Down: ", dcc.Input(id='down', value=1, type='number')]),
    html.Div(["Yards to Go: ", dcc.Input(id='ydstogo', value=10, type='number')]),
    html.Div(["Roof: ", dcc.Input(id='roof', value="retractable", type='text')]),
    html.Br(),
    html.Div(id='Touchdown'),
    html.Div(id='Opp_Touchdown'),
    html.Div(id='Field_Goal'),
    html.Div(id='Opp_Field_Goal'),
    html.Div(id='Safety'),
    html.Div(id='Opp_Safety'),
    html.Div(id='No_Score')
], style={'color': 'white'})


@callback(
    Output(component_id='Touchdown', component_property='children'),
    Output(component_id='Opp_Touchdown', component_property='children'),
    Output(component_id='Field_Goal', component_property='children'),
    Output(component_id='Opp_Field_Goal', component_property='children'),
    Output(component_id='Safety', component_property='children'),
    Output(component_id='Opp_Safety', component_property='children'),
    Output(component_id='No_Score', component_property='children'),
    Input(component_id='season', component_property='value'),
    Input(component_id='home_team', component_property='value'),
    Input(component_id='away_team', component_property='value'),
    Input(component_id='posteam', component_property='value'),
    Input(component_id='defteam', component_property='value'),
    Input(component_id='posteam_timeouts_remaining', component_property='value'),
    Input(component_id='defteam_timeouts_remaining', component_property='value'),
    Input(component_id='half_seconds_remaining', component_property='value'),
    Input(component_id='yardline_100', component_property='value'),
    Input(component_id='down', component_property='value'),
    Input(component_id='ydstogo', component_property='value'),
    Input(component_id='roof', component_property='value')
)
def get_epa_predictions(season, home_team, away_team, posteam, defteam, posteam_timeouts_remaining, defteam_timeouts_remaining, half_seconds_remaining, yardline_100, down, ydstogo, roof):
    req = requests.post(ml_serving_url + epa_path, json={
        "season": [season],
        "home_team": [home_team],
        "away_team": [away_team],
        "posteam": [posteam],
        "defteam": [defteam],
        "posteam_timeouts_remaining": [posteam_timeouts_remaining],
        "defteam_timeouts_remaining": [defteam_timeouts_remaining],
        "half_seconds_remaining": [half_seconds_remaining],
        "yardline_100": [yardline_100],
        "down": [down],
        "ydstogo": [ydstogo],
        "roof": [roof]
    })

    y = req.json()['y'][0]
    return (f"Touchdown: {y['Touchdown']}", f"Opp_Touchdown: {y['Opp_Touchdown']}", f"Field_Goal: {y['Field_Goal']}", f"Opp_Field_Goal: {y['Opp_Field_Goal']}", f"Safety: {y['Safety']}", f"Opp_Safety: {y['Opp_Safety']}", f"No_Score: {y['No_Score']}")

@callback(
    Output(component_id='home_win_prob', component_property='children'),
    Output(component_id='away_win_prob', component_property='children'),
    Input(component_id='game_id', component_property='value'),
    Input(component_id='home_team_wp', component_property='value'),
    Input(component_id='away_team_wp', component_property='value'),
    Input(component_id='posteam_wp', component_property='value'),
    Input(component_id='defteam_wp', component_property='value'),
    Input(component_id='posteam_timeouts_remaining_wp', component_property='value'),
    Input(component_id='defteam_timeouts_remaining_wp', component_property='value'),
    Input(component_id="game_seconds_remaining", component_property='value'),
    Input(component_id='half_seconds_remaining_wp', component_property='value'),
    Input(component_id='yardline_100_wp', component_property='value'),
    Input(component_id='down_wp', component_property='value'),
    Input(component_id='ydstogo_wp', component_property='value'),
    Input(component_id='spread_line', component_property='value'),
    Input(component_id='score_differential', component_property='value'),
    Input(component_id='qtr', component_property='value'),
    Input(component_id='result', component_property='value')
)
def get_vegas_wp_predictions(game_id, home_team_wp, away_team_wp, posteam_wp, defteam_wp, posteam_timeouts_remaining_wp, defteam_timeouts_remaining_wp, game_seconds_remaining, half_seconds_remaining_wp, yardline_100_wp, down_wp, ydstogo_wp, spread_line, score_differential, qtr, result): 
    req = requests.post(ml_serving_url + wp_path, json={
        "game_id": [game_id],
        "home_team": [home_team_wp],
        "away_team": [away_team_wp],
        "posteam": [posteam_wp],
        "defteam": [defteam_wp],
        "posteam_timeouts_remaining": [posteam_timeouts_remaining_wp],
        "defteam_timeouts_remaining": [defteam_timeouts_remaining_wp],
        "game_seconds_remaining": [game_seconds_remaining],
        "half_seconds_remaining": [half_seconds_remaining_wp],
        "yardline_100": [yardline_100_wp],
        "down": [down_wp],
        "ydstogo": [ydstogo_wp],
        "spread_line": [spread_line],
        "score_differential": [score_differential],
        "qtr": [qtr],
        "result": [result]
    })

    y = req.json()['y'][0]
    return (f"{home_team_wp} Win Probability: {y[0]}", f"{away_team_wp} Win Probability: {y[1]}")

app.run(debug=True)

In [1]:
from matplotlib import pyplot as plt
import matplotlib.ticker as plticker
import itertools
import pandas as pd
import numpy as np


WP_MODEL_FEATURES = [
    "home_team",
    "away_team",
    "posteam",
    "defteam",
    "yardline_100",
    "half_seconds_remaining",
    "game_seconds_remaining",
    "game_half",
    "down",
    "ydstogo",
    "home_timeouts_remaining",
    "away_timeouts_remaining",
    "score_differential_post"
]


def get_epa(df, mode, play_type):
    return df.loc[df[play_type] == 1, :].groupby([mode, 'season', 'week'], as_index=False)['epa'].mean()


def plot_passing_epa(df, team):
    tm = df.loc[df['team'] == team, :].assign(
        season_week=lambda x: 'w' + x.week.astype(str) + ' (' + x.season.astype(str) + ')'
    ).set_index('season_week')
    fig, ax = plt.subplots()
    loc = plticker.MultipleLocator(base=16)
    ax.xaxis.set_major_locator(loc)
    ax.tick_params(axis='x', rotation=75)
    ax.plot(tm['epa_shifted_passing_offense'], lw=1, alpha=0.5)
    ax.plot(tm['ewma_dynamic_window_passing_offense'], lw=2)
    ax.plot(tm['ewma_passing_offense'], lw=2);
    plt.axhline(y=0, color='red', lw=1.5, alpha=0.5)
    ax.legend(['Passing EPA', 'EWMA on EPA with dynamic window', 'Static 10 EWMA on EPA'])
    ax.set_title(f'{team} Passing EPA per play')
    return plt


def dynamic_window_ewma(x):
    """
    Calculate rolling exponentially weighted EPA with a dynamic window size
    """
    values = np.zeros(len(x))
    for i, (_, row) in enumerate(x.iterrows()):
        epa = x.epa_shifted[:i + 1]
        if row.week > 10:
            values[i] = epa.ewm(min_periods=1, span=row.week).mean().values[-1]
        else:
            values[i] = epa.ewm(min_periods=1, span=10).mean().values[-1]

    return pd.Series(values, index=x.index)


def lag_epa_one_period(df, mode):
    return df.groupby(mode)['epa'].shift()


def calculate_ewma(df, mode):
    return df.groupby(mode)['epa_shifted'].transform(lambda x: x.ewm(min_periods=1, span=10).mean())


def calculate_dynamic_window_ewma(df, mode):
    return df.groupby(mode).apply(dynamic_window_ewma).values


def merge_epa_data(rush_df, pass_df, mode):
    return rush_df.merge(pass_df, on=[mode, 'season', 'week'], suffixes=('_rushing', '_passing')).rename(
        columns={mode: 'team'})


def get_schedule(dataset_df, epa_df):
    schedule = dataset_df[
        ['season', 'week', 'home_team', 'away_team', 'home_score',
         'away_score']].drop_duplicates().reset_index(
        drop=True).assign(
        home_team_win=lambda x: (x.home_score > x.away_score).astype(int))
    return schedule.merge(epa_df.rename(columns={'team': 'home_team'}), on=['home_team', 'season', 'week']).merge(
        epa_df.rename(columns={'team': 'away_team'}), on=['away_team', 'season', 'week'], suffixes=('_home', '_away'))


def get_most_important_features(df):
    features = [column for column in df.columns if 'ewma' in column and 'dynamic' in column]
    return features


def ewma(data, window):
    """
    Calculate the most recent value for EWMA given an array of data and a window size
    """
    alpha = 2 / (window + 1.0)
    alpha_rev = 1 - alpha
    scale = 1 / alpha_rev
    n = data.shape[0]
    r = np.arange(n)
    scale_arr = scale ** r
    offset = data[0] * alpha_rev ** (r + 1)
    pw0 = alpha * alpha_rev ** (n - 1)
    mult = data * pw0 * scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums * scale_arr[::-1]
    return out[-1]


def get_pregame_predictions(df, season, model, features, bet_lines_df):
    game_df = df.loc[(df['season'] == season)]
    pregame_data = pd.DataFrame()
    for i, x in bet_lines_df.iterrows():
        home = x['home_team']
        away = x['away_team']
        offense = game_df.loc[(game_df['posteam'] == home) | (game_df['posteam'] == away)]
        defense = game_df.loc[(game_df['defteam'] == home) | (game_df['defteam'] == away)]
        rushing_offense = offense.loc[offense['rush_attempt'] == 1].groupby(['posteam', 'week'], as_index=False)[
            'epa'].mean().rename(columns={'posteam': 'team'})
        passing_offense = offense.loc[offense['pass_attempt'] == 1].groupby(['posteam', 'week'], as_index=False)[
            'epa'].mean().rename(columns={'posteam': 'team'})
        rushing_defense = defense.loc[defense['rush_attempt'] == 1].groupby(['defteam', 'week'], as_index=False)[
            'epa'].mean().rename(columns={'defteam': 'team'})
        passing_defense = defense.loc[defense['pass_attempt'] == 1].groupby(['defteam', 'week'], as_index=False)[
            'epa'].mean().rename(columns={'defteam': 'team'})
        pregame = np.zeros(8)

        for i, (tm, stat_df) in enumerate(
                itertools.product([home, away], [rushing_offense, passing_offense, rushing_defense, passing_defense])):
            ewma_value = ewma(stat_df.loc[stat_df['team'] == tm]['epa'].values, 20)
            pregame[i] = ewma_value

        pregame_df = pd.DataFrame()
        for x, y in zip(features, pregame):
            pregame_df[x] = [y]
        pregame_data = pd.concat([pregame_data, pregame_df], ignore_index=True)
    pregame_matrix = xgb.DMatrix(data=pregame_data, nthread=-1)
    bet_lines_df['home_win_prob'] = pd.DataFrame(model.predict(pregame_matrix),
                                                 columns=['home_win_prob'])
    bet_lines_df["final_score_diff"] = bet_lines_df["home_score"] - bet_lines_df["away_score"]
    bet_lines_df["recommended_bet"] = np.where(
        (bet_lines_df['home_win_prob'] > 0.6) & (
                bet_lines_df['home_spread_line'] > 0), bet_lines_df["home_team"], bet_lines_df["away_team"])
    bet_lines_df["final_spread"] = bet_lines_df["home_spread_line"] + bet_lines_df["final_score_diff"]
    bet_lines_df["correct_prediction"] = np.where(
        (bet_lines_df["recommended_bet"] == bet_lines_df["home_team"]) & (bet_lines_df["final_spread"] >= 0) | (
                bet_lines_df["recommended_bet"] == bet_lines_df["away_team"]) & (bet_lines_df["final_spread"] < 0), 1,
        0)
    return bet_lines_df

In [2]:
import nfl_data_py as nfl
import xgboost as xgb

seasons = list(range(1999, 2024, 1))

pbp_data = nfl.import_pbp_data(seasons, thread_requests=True)

# seperate EPA in to rushing offense, rushing defense, passing offense, passing defense for each team
rushing_offense_epa = get_epa(pbp_data, 'posteam', 'rush_attempt')
rushing_defense_epa = get_epa(pbp_data, 'defteam', 'rush_attempt')
passing_offense_epa = get_epa(pbp_data, 'posteam', 'pass_attempt')
passing_defense_epa = get_epa(pbp_data, 'defteam', 'pass_attempt')

# lag EPA one period back
rushing_offense_epa['epa_shifted'] = lag_epa_one_period(rushing_offense_epa, 'posteam')
rushing_defense_epa['epa_shifted'] = lag_epa_one_period(rushing_defense_epa, 'defteam')
passing_offense_epa['epa_shifted'] = lag_epa_one_period(passing_offense_epa, 'posteam')
passing_defense_epa['epa_shifted'] = lag_epa_one_period(passing_defense_epa, 'defteam')

# In each case, calculate EWMA with a static window and dynamic window and assign it as a column
rushing_offense_epa['ewma'] = calculate_ewma(rushing_offense_epa, 'posteam')
rushing_offense_epa['ewma_dynamic_window'] = calculate_dynamic_window_ewma(rushing_offense_epa, 'posteam')

rushing_defense_epa['ewma'] = calculate_ewma(rushing_defense_epa, 'defteam')
rushing_defense_epa['ewma_dynamic_window'] = calculate_dynamic_window_ewma(rushing_defense_epa, 'defteam')

passing_offense_epa['ewma'] = calculate_ewma(passing_offense_epa, 'posteam')
passing_offense_epa['ewma_dynamic_window'] = calculate_dynamic_window_ewma(passing_offense_epa, 'posteam')

passing_defense_epa['ewma'] = calculate_ewma(passing_defense_epa, 'defteam')
passing_defense_epa['ewma_dynamic_window'] = calculate_dynamic_window_ewma(passing_defense_epa, 'defteam')

# Merge all the data together
offense_epa = merge_epa_data(rushing_offense_epa, passing_offense_epa, 'posteam')
defense_epa = merge_epa_data(rushing_defense_epa, passing_defense_epa, 'defteam')

epa = offense_epa.merge(defense_epa, on=['team', 'season', 'week'], suffixes=('_offense', '_defense'))

# remove the first season of data
epa = epa.loc[epa['season'] != epa['season'].unique()[0], :]

epa = epa.reset_index(drop=True)
model = xgb.Booster()
model.load_model('ewma_model.json')

schedule = get_schedule(pbp_data, epa)
features = get_most_important_features(schedule)

Downcasting floats.


In [8]:
preseason_games = pd.DataFrame({
    'home_team': ['HOU', 'KC', 'DAL', 'DET', 'TB', 'BUF', 'BAL', 'SF', 'DET', 'BUF', 'BAL', 'SF'],
    'away_team': ['CLE', 'MIA', 'GB', 'LA', 'PHI', 'PIT', 'HOU', 'GB', 'TB', 'KC', 'KC', 'DET'],
    'home_score': [45, 26, 32, 24, 32, 31, 34, 24, 31, 24, 0, 0],
    'away_score': [14, 7, 48, 23, 9, 17, 10, 21, 23, 27, 0, 0],
    'home_spread_line': [2, -4.5, -7.5, -3, 3, 10, -8.5, -10, -6, -2.5, -3.5, -7]})
pregame_predictions = get_pregame_predictions(pbp_data, 2023, model, features, preseason_games)
pregame_predictions

,home_team,away_team,home_score,away_score,home_spread_line,home_win_prob,final_score_diff,recommended_bet,final_spread,correct_prediction
0,HOU,CLE,45,14,2.0,0.628303,31,HOU,33.0,1
1,KC,MIA,26,7,-4.5,0.393549,19,MIA,14.5,0
2,DAL,GB,32,48,-7.5,0.656917,-16,GB,-23.5,1
3,DET,LA,24,23,-3.0,0.631059,1,LA,-2.0,1
4,TB,PHI,32,9,3.0,0.698613,23,TB,26.0,1
5,BUF,PIT,31,17,10.0,0.762121,14,BUF,24.0,1
6,BAL,HOU,34,10,-8.5,0.705156,24,HOU,15.5,0
7,SF,GB,24,21,-10.0,0.589951,3,GB,-7.0,1
8,DET,TB,31,23,-6.0,0.675337,8,TB,2.0,0
9,BUF,KC,24,27,-2.5,0.669713,-3,KC,-5.5,1


In [4]:
print(f"Accuracy: {round(pregame_predictions['correct_prediction'].mean() * 100, 2)}%")

Accuracy: 70.0%


In [3]:
import nfl_data_py as nfl

schedule_df = nfl.import_schedules([2023])
schedule_df.sort_values(by=["gameday", "gametime"], inplace=True)
schedule_df

,game_id,season,game_type,week,gameday,weekday,gametime,away_team,away_score,home_team,...,wind,away_qb_id,home_qb_id,away_qb_name,home_qb_name,away_coach,home_coach,referee,stadium_id,stadium
6421,2023_01_DET_KC,2023,REG,1,2023-09-07,Thursday,20:20,DET,21.0,KC,...,NaN,00-0033106,00-0033873,Jared Goff,Patrick Mahomes,Dan Campbell,Andy Reid,John Hussey,KAN00,GEHA Field at Arrowhead Stadium
6422,2023_01_CAR_ATL,2023,REG,1,2023-09-10,Sunday,13:00,CAR,10.0,ATL,...,NaN,00-0039150,00-0038122,Bryce Young,Desmond Ridder,Frank Reich,Arthur Smith,Brad Rogers,ATL97,Mercedes-Benz Stadium
6423,2023_01_HOU_BAL,2023,REG,1,2023-09-10,Sunday,13:00,HOU,9.0,BAL,...,NaN,00-0039163,00-0034796,C.J. Stroud,Lamar Jackson,DeMeco Ryans,John Harbaugh,Tra Blake,BAL00,M&T Bank Stadium
6424,2023_01_CIN_CLE,2023,REG,1,2023-09-10,Sunday,13:00,CIN,3.0,CLE,...,NaN,00-0036442,00-0033537,Joe Burrow,Deshaun Watson,Zac Taylor,Kevin Stefanski,Clete Blakeman,CLE00,FirstEnergy Stadium
6425,2023_01_JAX_IND,2023,REG,1,2023-09-10,Sunday,13:00,JAX,31.0,IND,...,NaN,00-0036971,00-0039164,Trevor Lawrence,Anthony Richardson,Doug Pederson,Shane Steichen,Clay Martin,IND00,Lucas Oil Stadium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6698,2023_19_PHI_TB,2023,WC,19,2024-01-15,Monday,20:00,PHI,9.0,TB,...,2.0,00-0036389,00-0034855,Jalen Hurts,Baker Mayfield,Nick Sirianni,Todd Bowles,Adrian Hill,TAM00,Raymond James Stadium
6699,2023_20_HOU_BAL,2023,DIV,20,2024-01-20,Saturday,16:30,HOU,NaN,BAL,...,NaN,00-0039163,00-0034796,C.J. Stroud,Lamar Jackson,DeMeco Ryans,John Harbaugh,NaN,BAL00,M&T Bank Stadium
6700,2023_20_GB_SF,2023,DIV,20,2024-01-20,Saturday,20:00,GB,NaN,SF,...,NaN,00-0036264,00-0037834,Jordan Love,Brock Purdy,Matt LaFleur,Kyle Shanahan,NaN,SFO01,Levi's Stadium
6701,2023_20_TB_DET,2023,DIV,20,2024-01-21,Sunday,15:00,TB,NaN,DET,...,NaN,00-0034855,00-0033106,Baker Mayfield,Jared Goff,Todd Bowles,Dan Campbell,NaN,DET00,Ford Field


In [ ]:
import requests

schedule_df = nfl.import_schedules([2023])
schedule

api_key = "178a47c0d06b6e8243357dcaf3aa0cf8"
region = "us"
markets = "h2h,spreads,totals"
date = "2021-09-"
odds_api_url = "https://api.the-odds-api.com"
historical_odds = "/v4/historical/sports/americanfootball_nfl/odds"
req = requests.get(odds_api_url + historical_odds, params={"apiKey": api_key, "region": "us", "sport": "americanfootball_nfl", "mkt": "spreads", "dateFormat": "iso", "oddsFormat": "american", "from": "2021-09-09", "to": "2021-09-13"})

In [7]:
from dash import Dash, dcc, html, Input, Output, callback, dash_table
import requests

app = Dash(__name__)

ml_serving_url = "https://clearml-serving.internal.magiccityit.com/serve"
epa_path = "/epa_model/"
wp_path = "/vegas_wp_model/"

easy_home_team = 'DET'
easy_away_team = 'SF'
easy_spread_line = -7

input_table = html.Table([
    html.Tr([
        html.Td(html.Label('Season:')),
        html.Td(dcc.Input(type='number', value=2023, id='season', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Game ID:')),
        html.Td(dcc.Input(type='text', value=f"2023_{easy_away_team}_{easy_home_team}", id='game_id', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Home Team:')),
        html.Td(dcc.Input(type='text', value=easy_home_team, id='home_team', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Away Team:')),
        html.Td(dcc.Input(type='text', value=easy_away_team, id='away_team', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Home Score:')),
        html.Td(dcc.Input(type='number', value=0, id='home_score', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Away Score:')),
        html.Td(dcc.Input(type='number', value=0, id='away_score', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Possessing Team:')),
        html.Td(dcc.Input(type='text', value=easy_home_team, id='posteam', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Defending Team:')),
        html.Td(dcc.Input(type='text', value=easy_away_team, id='defteam', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Possessing Team Timeouts Remaining:')),
        html.Td(dcc.Input(type='number', value=3, id='posteam_timeouts_remaining', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Defending Team Timeouts Remaining:')),
        html.Td(dcc.Input(type='number', value=3, id='defteam_timeouts_remaining', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Seconds Remaining in the Game:')),
        html.Td(dcc.Input(type='number', value=3600, id='game_seconds_remaining', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Seconds Remaining this Half:')),
        html.Td(dcc.Input(type='number', value=1800, id='half_seconds_remaining', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Quarter:')),
        html.Td(dcc.Input(type='number', value=1, id='qtr', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Yardline Position:')),
        html.Td(dcc.Input(type='number', value=75, id='yardline_100', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Down:')),
        html.Td(dcc.Input(type='number', value=1, id='down', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Yards to Go:')),
        html.Td(dcc.Input(type='number', value=10, id='ydstogo', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Spread Line:')),
        html.Td(dcc.Input(type='number', value=easy_spread_line, id='spread_line', debounce=True))
    ]),
    html.Tr([
        html.Td(html.Label('Result:')),
        html.Td(dcc.Input(type='number', value=0, id='result', debounce=True))
    ])
])

output_table = html.Table([
    html.Tr([
        html.Td(html.Div(id="home_team_result")),
        html.Td(html.Div(id='home_win_prob'))
    ]),
    html.Tr([
        html.Td(html.Div(id="away_team_result")),
        html.Td(html.Div(id='away_win_prob'))
    ])
])

app.layout = html.Div([
    html.H2("Spread-Based Win Probability Calculator"),
    input_table,
    html.Br(),
    html.H2("Predicted Probabilities"),
    output_table    
], style={'color': 'white'})

@callback(
    [Output(component_id='home_win_prob', component_property='children'),
    Output(component_id='away_win_prob', component_property='children'),
    Output(component_id='home_team_result', component_property='children'),
    Output(component_id='away_team_result', component_property='children')],
    [Input(component_id='season', component_property='value'),
    Input(component_id='game_id', component_property='value'),
    Input(component_id='home_team', component_property='value'),
    Input(component_id='away_team', component_property='value'),
    Input(component_id='home_score', component_property='value'),
    Input(component_id='away_score', component_property='value'),
    Input(component_id='posteam', component_property='value'),
    Input(component_id='defteam', component_property='value'),
    Input(component_id='posteam_timeouts_remaining', component_property='value'),
    Input(component_id='defteam_timeouts_remaining', component_property='value'),
    Input(component_id="game_seconds_remaining", component_property='value'),
    Input(component_id='half_seconds_remaining', component_property='value'),
    Input(component_id='yardline_100', component_property='value'),
    Input(component_id='down', component_property='value'),
    Input(component_id='ydstogo', component_property='value'),
    Input(component_id='spread_line', component_property='value'),
    Input(component_id='qtr', component_property='value'),
    Input(component_id='result', component_property='value')]
)
def get_vegas_wp_predictions(season, game_id, home_team, away_team, home_score, away_score, posteam, defteam, posteam_timeouts_remaining, defteam_timeouts_remaining, game_seconds_remaining, half_seconds_remaining, yardline_100, down, ydstogo, spread_line, qtr, result):
    score_differential = home_score - away_score
    req = requests.post(ml_serving_url + wp_path, json={
        "game_id": [game_id],
        "home_team": [home_team],
        "away_team": [away_team],
        "posteam": [posteam],
        "defteam": [defteam],
        "posteam_timeouts_remaining": [posteam_timeouts_remaining],
        "defteam_timeouts_remaining": [defteam_timeouts_remaining],
        "game_seconds_remaining": [game_seconds_remaining],
        "half_seconds_remaining": [half_seconds_remaining],
        "yardline_100": [yardline_100],
        "down": [down],
        "ydstogo": [ydstogo],
        "spread_line": [spread_line],
        "score_differential": [score_differential],
        "qtr": [qtr],
        "result": [result]
    })
    df = pd.DataFrame({"home_team": [home_team], "away_team": [away_team], "home_score": [home_score], "away_score": [away_score], "home_spread_line": [spread_line]})
    preds = get_pregame_predictions(pbp_data, season, model, features, df)
    preds['away_win_prob'] = 1 - preds['home_win_prob']
    y = req.json()['y'][0]
    home_ewma_norm = preds['home_win_prob'].tolist()[0] / (preds['home_win_prob'].tolist()[0] + preds['away_win_prob'].tolist()[0])
    away_ewma_norm = preds['away_win_prob'].tolist()[0] / (preds['home_win_prob'].tolist()[0] + preds['away_win_prob'].tolist()[0])
    home_win_prob = (y[1] * home_ewma_norm) / ((y[1] * home_ewma_norm) + (y[0] * away_ewma_norm))
    away_win_prob = (y[0] * away_ewma_norm) / ((y[1] * home_ewma_norm) + (y[0] * away_ewma_norm))
    return f"{round(home_win_prob * 100)}%", f"{round(away_win_prob * 100)}%", f"{home_team} Win Probability", f"{away_team} Win Probability"
    # y = req.json()['y'][0]
    # return f"{round(y[0] * 100)}%", f"{round(y[1] * 100)}%", f"{home_team} Win Probability", f"{away_team} Win Probability"

app.run(debug=True)
